# Asyncio from scratch

In [ ]:
import collections
import random


def coroutine(func):
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen

    return wrapper


class Task:
    task_id = 0

    def __init__(self, coro):
        self.coro = coro
        Task.task_id += 1
        self.id = Task.task_id
        self.name = f"{coro.__name__}_{self.id}"
        self.status = "READY"


class Scheduler:
    def __init__(self):
        self.ready = collections.deque()
        self.tasks = {}

    def new(self, coro):
        task = Task(coro)
        self.tasks[task.name] = task
        self.schedule(task)
        return task

    def schedule(self, task):
        self.ready.append(task)

    def run(self):
        while self.ready:
            task = self.ready.popleft()
            task.status = "RUNNING"
            try:
                next(task.coro)
                if task.status == "RUNNING":
                    self.schedule(task)
            except StopIteration:
                task.status = "TERMINATED"

            print(f"Active tasks: {list(self.tasks.keys())}")
            print(f"Ready queue: {[t.name for t in self.ready]}")
            print()


@coroutine
def async_task(name):
    for i in range(3):
        print(f"{name}: Starting step {i}")
        yield from async_sleep(random.randint(1, 3))
        print(f"{name}: Completed step {i}")


@coroutine
def async_sleep(duration):
    print(f"Sleeping for {duration} time units")
    for _ in range(duration):
        yield


scheduler = Scheduler()
scheduler.new(async_task("Task A"))
scheduler.new(async_task("Task B"))
scheduler.new(async_task("Task C"))
scheduler.run()
